In [1]:
from __future__ import annotations

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

from helper import *

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


# Intro

# Load Data

In [3]:
# Reduced data to make it manageable for smaller systems
DATA_FILE: str = '../data/shakespeare_small.txt'

In [4]:
with open(DATA_FILE, 'r') as data_file:
    raw_text = data_file.read()

In [5]:
print(f'Number of characters: {len(raw_text):,}')

Number of characters: 50,085


# Character-Based Generation

## Encode Text into Integer Tokens

### Normalization

> - Skip?
> - lowercase?

In [6]:
def normalize_text(text: str) -> str:
    # Can be multiple actions to normalize text
    normalized_text = text.lower()
    return normalized_text

In [7]:
normalized_text = normalize_text(raw_text)

### Pretokenization

In [8]:
def pretokenize_text(text: str) -> list[str]:
    smaller_pieces = [char for char in text]
    return smaller_pieces

In [9]:
shakespeare_text = pretokenize_text(normalized_text)

### Tokenize

In [10]:
character_mapping = TokenMapping(shakespeare_text)

In [11]:
encoded = [
    character_mapping.token2index[char]
    for char in shakespeare_text
]

### Postprocessing

> Skip

## Prepare Dataset

In [12]:
n_tokens = character_mapping.n_tokens
dataset_size = len(encoded)

In [13]:
length = 32 # Number of characters
batch_size = 32

train_dataset = ShakespeareDataset(encoded, length)
train_loader = DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=batch_size,
)

## Define Model

In [14]:
model = build_model(n_tokens)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

## Define Text Genaration

In [15]:
def generate_text(
    model,
    input_str: str,
    num_chars: int = 100,
    temperature: float = 1.0,
) -> str:
    model.eval()
    generated_text = input_str.lower()
    input_tensor = tokens_to_index_tensor(
        tokens=input_str.lower(),
        token_index_mapping=character_mapping.token2index,
    )
    
    with torch.no_grad():
        for _ in range(num_chars):
            output = model(input_tensor.to(device))
            probabilities = nn.functional.softmax(
                output[0, -1] / temperature,
                dim=0,
            )
            next_char_idx = torch.multinomial(probabilities, 1).item()
            generated_text += character_mapping.index2token[next_char_idx]
            input_tensor = torch.cat(
                [
                    input_tensor,
                    torch.tensor([[next_char_idx]], dtype=torch.long),
                ],
                1,
            )
            
    return generated_text

## Train Model

In [16]:
PHRASE = 'To be or not to be'
epochs = 5

start = start_time()
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch.to(device))
        loss = criterion(output.transpose(1, 2), y_batch.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}')
    print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / epochs * 100, loss))
    print('-'*72)
    gen_output = generate_text(model, PHRASE, 100)
    print(gen_output)

Epoch 1/5, Loss: 2.264691512500897
 6.4s (0 0%) 2.1262]
------------------------------------------------------------------------
to be or not to be to a u;s: won.

lantherat in iide. hate.

fond a wist the dcith ingle.

sifkersy noles of a,
boy i 
Epoch 2/5, Loss: 1.9142016943270406
 12.9s (1 20%) 1.6515]
------------------------------------------------------------------------
to be or not to beemeneting; to hercesary. thinetry dift dit the now verfens:
ansted i do gatior, yet marcius:
that ga
Epoch 3/5, Loss: 1.829020601948991
 19.3s (2 40%) 1.6953]
------------------------------------------------------------------------
to be or not to bear bele being dto shal ot
he coprich
i gok fulk ane heqhy;
will hearinius sessidius:
whis to stave h
Epoch 4/5, Loss: 1.7857095245355235
 26.1s (3 60%) 1.5915]
------------------------------------------------------------------------
to be or not to beod store would on me, fands!
ip of he orere cimines:
to soctreaber.

moneselfour thoust leat, i him,


## Generate Text

In [17]:
output = generate_text(model, 'To be or not to be', 100)
print(output)

to be or not to bepetince this ance,
fut, and in hine
the comperce the my hl't buther, wherety,
speats the mores, and 


# Token-Based Generation

## Encode Text into Integer Tokens

### Tokenize (Choose)

In [18]:
# Choose a pretrained tokenizer to use
xlmr_model_name = 'xlm-roberta-base'
bert_model_name = 'bert-base-cased'
bert_model_name_uncased = 'bert-base-uncased'

bert_uncased_tokenizer = AutoTokenizer.from_pretrained(
    bert_model_name_uncased,
)

In [19]:
shakespeare_text = (
    tokenize_text(
        tokenizer=bert_uncased_tokenizer,
        text=raw_text,
    )
)

In [20]:
token_mapping = TokenMapping(shakespeare_text)

In [21]:
encoded = [
    token_mapping.token2index[char]
    for char in shakespeare_text
]

## Prepare Dataset

In [22]:
n_tokens = token_mapping.n_tokens
dataset_size = len(encoded)

In [23]:
length = 16  # Tokens 
batch_size = 32

train_dataset = ShakespeareDataset(encoded, length)
train_loader = DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=batch_size,
)

## Define Model

In [24]:
model = build_model(n_tokens)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

## Define Text Genaration

In [25]:
def generate_text(
    tokenizer,
    model,
    input_str: str,
    num_tokens: int = 100,
    temperature: float = 1.0,
) -> str:
    model.eval()
    tokenized_text = tokenize_text(tokenizer=tokenizer, text=input_str)
    input_tensor = tokens_to_index_tensor(
        tokens=tokenized_text,
        token_index_mapping=token_mapping.token2index,
    )

    generated_text = []
    with torch.no_grad():
        for _ in range(num_tokens):
            output = model(input_tensor.to(device))
            probabilities = nn.functional.softmax(
                output[0, -1] / temperature,
                dim=0,
            )
            next_token_idx = torch.multinomial(probabilities, 1).item()
            generated_text.append(token_mapping.index2token[next_token_idx])
            input_tensor = torch.cat(
                [
                    input_tensor,
                    torch.tensor([[next_token_idx]], dtype=torch.long),
                ],
                1,
            )
    # Convert to text again
    output_ids = tokenizer.convert_tokens_to_ids(generated_text)
    output_str = input_str + ' ' + tokenizer.decode(output_ids)
    return output_str

## Train Model

In [26]:
epochs = 5

start = start_time()
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch.to(device))
        loss = criterion(output.transpose(1, 2), y_batch.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}')
    print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / epochs * 100, loss))
    print('-'*72)
    output = generate_text(bert_uncased_tokenizer, model, 'To be or not to be', 30)
    print(output)


Epoch 1/5, Loss: 6.16962888351032
 2.8s (0 0%) 5.1410]
------------------------------------------------------------------------
To be or not to be ##her south let. : br soldier speak while'oak run some you assembly that ; good they you! hat girons beingach in.rio rather
Epoch 2/5, Loss: 5.364991634721593
 5.6s (1 20%) 4.9285]
------------------------------------------------------------------------
To be or not to be ##uss let hearts. aidi i will retire of have, a goo rather,, to of have ; behind mills. far thousands they artpt
Epoch 3/5, Loss: 4.842905401893486
 8.4s (2 40%) 4.1508]
------------------------------------------------------------------------
To be or not to be pray choice of slain, which remain i'll not threeers! theyinius : tell express vols to relieve fools a : nos, who
Epoch 4/5, Loss: 4.4670529794809015
 11.2s (3 60%) 4.1122]
------------------------------------------------------------------------
To be or not to be des sp subjects have first citizen : less's er patric

## Generate Text

In [27]:
output = generate_text(bert_uncased_tokenizer, model, 'To be or not to be', 30)
print(output)

To be or not to be carrythling can's their wholesomeaves, which like you : alone forth, his brows honour me the gentry horse a net, him alone


# Comparison Between Generation